In [2]:
import os
import polars as pl
import altair as alt

In [4]:
import warnings
from src.kristi_promin import kristi_promin
from src.me_to_neurazi import me_to_neurazi
from src.alt_friendly import alt_friendly

pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

In [6]:
df = pl.scan_parquet(
    "data/projevy.parquet"
).filter(
    pl.col('datum').dt.year() >= 1993
).filter(
    pl.col("komora_komplet").str.contains('Poslanecká') | pl.col('komora_komplet').str.contains('PČR, PS')
).with_columns(
    pl.col('mluvci_id').str.split('/sqw/detail.sqw?id=').alias('id')
).with_columns(
    pl.when(pl.col('id').len() > 2).then(pl.col('id').list.slice(1)).alias('id')
).explode(
    "id"
).with_columns(
    pl.col("text").str.count_matches(" ").alias("pocet_slov") # tady budou zlobit dvojité mezery, pohlídat při čištění
).sort(
    by="datum"
).collect()

In [8]:
narust = {
    "pětikoalice":"(?i)[^\w]pětikoal",
    "energie, elektřina, plyn":"(?i)[^\w](energ[ie]|plyn|elekt[řr])",
    "korespondenční":"(?i)[^\w]korespondenčn",
#    "omikron":"(?i)omikron",
#    "HHC":"(?i)hhc",
#    "kratom":"(?i)kratom",
#    "drahota":"(?i)drahot",
#    "Green Deal":"(?i)green deal",
    "Ukrajina, Ukrajinci, ukrajinský":"(?i)[^\w]ukrajin",
    "Lex Něco/Někdo":"(?i)[^\w]lex \w{0,20}",
#    "Rusko, ruský":"(?i)[^\w]rusk",
#    "Křivoklát":"(?i)křivoklát",
    "menstruace":"(?i)[^\w]menstru[oau]",
#    "bitcoin":"(?i)bitco",
#    "kryptoměny":"(?i)kryptom",
    "bydlení":"(?i)[^\w](bydle[tn])",
#    "dozimetr":"(?i)dozimet",
    "inflace, zdražování":"(?i)[^\w](infla[cč]|zdraž)"
}

In [10]:
maxima = []
for p, n in narust.items():
    maximum = {
        'slovo':p,
        'max':
        df.filter(
            pl.col('obdobi') == 2021
        ).filter(
            pl.col('text').str.contains(n)
        ).with_columns(
            pl.col('text').str.count_matches(n).alias('pocet_vyskytu')
        ).group_by_dynamic(
            index_column='datum',every='1y'
        ).agg(
            pl.col('pocet_vyskytu').sum()
        ).select(
            pl.col('pocet_vyskytu').max()
        ).item()
    }
    maxima.append(maximum)

maxima = pl.DataFrame(maxima).sort(by='max',descending=True).select(pl.col('slovo')).to_series().to_list()

In [12]:
slov_po_roce = df.group_by_dynamic(index_column='datum',every='1y').agg(pl.col('pocet_slov').sum())
slov_po_ctvrtleti = df.group_by_dynamic(index_column='datum',every='3mo').agg(pl.col('pocet_slov').sum())

In [14]:
def cetnost_po_roce(popisek, hledani):
    return df.filter(
        pl.col('text').str.contains(hledani)
    ).with_columns(
        pl.col('text').str.count_matches(hledani).alias('pocet_vyskytu')
    ).group_by_dynamic(
        index_column='datum',every='1y'
    ).agg(
        pl.col('pocet_vyskytu').sum()
    ).join(
        slov_po_roce, on='datum', how='right'
    ).with_columns(
        (pl.col('pocet_vyskytu') / pl.col('pocet_slov')).alias('podil')
    ).select(
        pl.col(['datum','podil'])
    ).with_columns(
        pl.lit(popisek).alias('co')
    )

In [16]:
def cetnost_po_cvrtleti(popisek, hledani):
    return df.filter(
        pl.col('text').str.contains(hledani)
    ).with_columns(
        pl.col('text').str.count_matches(hledani).alias('pocet_vyskytu')
    ).group_by_dynamic(
        index_column='datum',every='3mo'
    ).agg(
        pl.col('pocet_vyskytu').sum()
    ).join(
        slov_po_ctvrtleti, on='datum', how='right'
    ).with_columns(
        (pl.col('pocet_vyskytu') / pl.col('pocet_slov')).alias('podil')
    ).select(
        pl.col(['datum','podil'])
    ).with_columns(
        pl.lit(popisek).alias('co')
    ).with_columns(
        pl.when(pl.col("datum") >= pl.date(2021,10,1)).then(pl.lit("současná Sněmovna")).otherwise(pl.lit("předchozí Sněmovny")).alias("ktera_snemovna")
    )

In [18]:
do_grafu = pl.DataFrame()
for popis, hled in narust.items():
    do_grafu = pl.concat([do_grafu,cetnost_po_cvrtleti(popis, hled)])

In [20]:
graf = alt.Chart(
    do_grafu.to_pandas(),
    title=alt.Title(
        ["O čem poslanci mluvili častěji než dřív"],
        subtitle=[
            "Kolik slov bylo ve Sněmovně zapotřebí k tomu, aby zazněl konkrétní výraz.",
            "Pozor na rozdílná měřítka: čím níže v grafu, tím vzácnější výskyt."
        ]
    ),
    width=330,
    height=50
).mark_bar(
    width=1
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None, tickCount=33, labelAngle=270)),
    alt.Y('podil:Q', axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6', labelExpr="datum.label == 0 ? '' : floor((1 / datum.label) + 0.5) + ' slov'",  tickCount=2, orient='right', title=None)),
    alt.Row('co:N', sort=maxima, title=None, header=alt.Header(labelAngle=0, labelAlign='left', labelAnchor='start', labelOrient='top', labelFont='Asap')),
    alt.Color("ktera_snemovna:N", 
              title=None,
              scale=alt.Scale(
                  range=['#6B8EAA','#ba4c43'
                        ]),
              legend=alt.Legend(
                  title=None,
                  direction="horizontal",
                  orient="top"
              )
    )
).configure_axisX(
    grid=False, domain=False
).configure_axisY(
    grid=True, domain=False
).configure_view(
    stroke='transparent'
).resolve_axis(
    x="independent",
    y="independent"
).resolve_scale(
    y="independent"
)

graf

alt.Chart(...)

In [ ]:
me_to_neurazi(
    graf=graf, 
    soubor='slova_skokani', 
    kredity='vizualizace: iROZHLAS.cz | zdroj projevů: psp.cz', 
    slozka_na_serveru='grafy-stenoprotokoly'
)